## Crop Data Merge
- This notebook merges FDW crop data of individual countries into a single data file with cross-checking "FNID" and "Name" between FDW data and FEWS NET's Shapefile.

In [1]:
import os, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_hdf
from tools import CreateLinkAdmin
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

### Merge crop data

In [2]:
# Merge crop data
countries_ISO = ['SO','MW','KE','BF','ML','TD','ZA','NE','ZM','AO','MZ']
container = []
for name in countries_ISO:
    df = pd.read_hdf('./data/crop/adm_crop_production_%s.hdf' % name)
    # Select the calibrated data
    df = df[df['gscd_code'] == 'calibrated']
    if name == 'KE':
        # Manual editing -------------------------- #
        # - Fill missing "Long" season records with "Annual" records for Kenya
        df_raw = df.copy()
        cols = [
            'fnid', 'country', 'country_code', 'admin_1', 'admin_2',
            'product', 'season_name', 'growing_month', 'harvest_month', 
            'crop_production_system', 'indicator'
        ]
        df['year'] = df['harvest_year']
        df = df.pivot_table(index='year', columns=cols, values='value')
        annual = df.loc[:,pd.IndexSlice[:,'Kenya',:,:,:,'Maize','Annual',:]]
        annual.columns = annual.columns.droplevel([1,2,3,4,5,6,7,8,9])
        long = df.loc[:,pd.IndexSlice[:,'Kenya',:,:,:,'Maize','Long',:]]
        long.columns = long.columns.droplevel([1,2,3,4,5,6,7,8,9])
        long = long.fillna(annual)
        df.loc[:,pd.IndexSlice[:,'Kenya',:,:,:,'Maize','Long',:]] = long.values

#        Only fill the long rains with annual values. Keep short rains as-are        
#        short = df.loc[:,pd.IndexSlice[:,'Kenya',:,:,:,'Maize','Short',:]]
#        short.columns = short.columns.droplevel([1,2,3,4,5,6,7,8,9])
#        short = short.fillna(annual)
#        df.loc[:,pd.IndexSlice[:,'Kenya',:,:,:,'Maize','Short',:]] = short.values
        df = df.T.stack(dropna=True).reset_index().rename(columns={0:'value'})
        # Restore columns such as growing year
        df = df.rename(columns={'year':'harvest_year'})
        df['gscd_code'] = 'calibrated'
        df['name'] = df['admin_1']
        df['growing_year'] = df['harvest_year']
        df.loc[df['season_name'] == 'Short', 'growing_year'] -= 1
        df = df[df_raw.columns]

        # ----------------------------------------- #
    container.append(df)
df = pd.concat(container, axis=0).reset_index(drop=True)

In [3]:
# Merge the latest shapefile
shape_container = []
path_dir = './data/shapefile/fewsnet/'
shape_container.append(gpd.read_file(os.path.join(path_dir, 'SO_Admin2_1990.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'MW_Admin2_2003.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'KE_Admin1_2013.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'BF_Admin2_2001.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'ML_Admin1_2016.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'TD_Admin1_2012.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'ZA_Admin1_1994.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'NE_Admin2_2012.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'ZM_Admin2_2011.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'MZ_Admin1_2013.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'AO_Admin1_2008.shp')))
adm_current = pd.concat(shape_container, axis=0).reset_index(drop=True)[['FNID','ADMIN0','ADMIN1','ADMIN2','geometry']]
adm_current = adm_current.to_crs(epsg=4326)
# - All processed shapefiles 
#shape_container.append(gpd.read_file(os.path.join(path_dir, 'MZ_Admin1_1990.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'UG_Admin2_2007.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'SS_Admin2_2011.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'CF_Admin2_2003.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'ZW_Admin2_2011.shp')))
shape_container.append(gpd.read_file(os.path.join(path_dir, 'GN_Admin2_1990.shp')))
adm_processed = pd.concat(shape_container, axis=0).reset_index(drop=True)[['FNID','ADMIN0','ADMIN1','ADMIN2','geometry']]
adm_processed = adm_processed.to_crs(epsg=4326)

# Update FNID to the latest FEWS NET's entire shapfile FNID (if different)
# For instance, MW2003A2 is the latest boundary in FDW data, but the FEWS NET's African ADMIN-2 shapefile contains MW2007A2 which is regarded as the same.
# We already use this African ADMIN shapefile to EO aggregation, so here we will just modify the FDW crop data.
adm1 = gpd.read_file('./data/shapefile/fewsnet/FEWSNET_Admin1.shp').to_crs("EPSG:4326")
adm2 = gpd.read_file('./data/shapefile/fewsnet/FEWSNET_Admin2.shp').to_crs("EPSG:4326")
# - Crop data
fnid_new = pd.concat([adm1.FNID, adm2.FNID], axis=0).reset_index(drop=True)
fnid_old = adm_current.FNID
diff = fnid_old[~fnid_old.isin(fnid_new)].values
diff_adm1 = diff[[t[7]=='1' for t in diff]]
diff_adm2 = diff[[t[7]=='2' for t in diff]]
old = adm_current[adm_current.FNID.isin(diff_adm1)]
new = adm1[adm1.ADMIN0.isin(old.ADMIN0.unique())]
link_adm1 = CreateLinkAdmin(new, old, 'ADMIN1', 'ADMIN1')[0]
old = adm_current[adm_current.FNID.isin(diff_adm2)]
new = adm2[adm2.ADMIN0.isin(old.ADMIN0.unique())]
link_adm2 = CreateLinkAdmin(new, old, 'ADMIN2', 'ADMIN2')[0]
rename_adm = {**link_adm1, **link_adm2}
df.fnid = df.fnid.replace(rename_adm)
# - Current shapefile
adm1_countries = adm_current[[t[7]=='1' for t in adm_current.FNID]].ADMIN0.unique()
adm2_countries = adm_current[[t[7]=='2' for t in adm_current.FNID]].ADMIN0.unique()
adm_current = pd.concat([adm1[adm1.ADMIN0.isin(adm1_countries)],
                         adm2[adm2.ADMIN0.isin(adm2_countries)],
                        ], axis=0).sort_values(by='FNID').reset_index(drop=True)

# Save files
save_hdf('./data/crop/adm_crop_production_ALL.hdf', df)
adm_current.to_file('./data/shapefile/adm_current.shp')
print('%s is saved.' % './data/shapefile/adm_current.shp')
adm_processed.to_file('./data/shapefile/adm_processed.shp')
print('%s is saved.' % './data/shapefile/adm_processed.shp')

./data/crop/adm_crop_production_ALL.hdf is saved.
./data/shapefile/adm_current.shp is saved.
./data/shapefile/adm_processed.shp is saved.


### [Public] Stable version to public folder

In [4]:
# Select specific countries 
country_list = ['Burkina Faso', 'Kenya', 'Somalia', 'Malawi']
stable_data = df[df['country'].isin(country_list)].reset_index(drop=True)
stable_shape = adm_current[adm_current['ADMIN0'].isin(country_list)].reset_index(drop=True)
stable_shape.geometry = stable_shape.geometry.simplify(0.001)
# Save files
fn_out = './public/gscd_data_stable.csv'
stable_data.to_csv(fn_out); print(f'{fn_out} is saved.')
fn_out = './public/gscd_shape_stable.shp'
stable_shape.to_file(fn_out); print(f'{fn_out} is saved.')
geojson = json.loads(stable_shape.to_json())
fn_out = './public/gscd_shape_stable.json'
with open(fn_out,'w') as f:
    f.write(json.dumps(geojson)); print(f'{fn_out} is saved.')

./public/gscd_data_stable.csv is saved.
./public/gscd_shape_stable.shp is saved.
./public/gscd_shape_stable.json is saved.


### Summary of "adm_crop_production_ALL.hdf"

In [5]:
shape = gpd.read_file('./data/shapefile/adm_current.shp')
df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
df = df.merge(shape[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
df = df.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
df = df[['fnid','country','admin1','admin2','product','season','harvest_month','harvest_year','indicator','value']]
cps = df[['country','product','season']].drop_duplicates().reset_index(drop=True)
df['admin'] = df['fnid'].apply(lambda x: x[2:8])
table = pd.DataFrame(
    index=pd.MultiIndex.from_frame(df[['country','admin','season']].drop_duplicates()),
    columns= cps['product'].unique(),
    data = '-'
).rename_axis(columns='product')
for i, (country_name, product_name, season_name) in cps.iterrows():
    fnids_country = shape.loc[shape['ADMIN0'] == country_name, 'FNID']
    sub = df[
        (df['country'] == country_name) &
        (df['product'] == product_name) &
        (df['season'] == season_name) &
        (df['indicator'] == 'area')
    ]
    count = sub.pivot_table(index='harvest_year', columns='fnid', values='value', aggfunc=len, fill_value=0)
    mean_count = count.sum().mean()
    string = '%d (%d/%d)' %  (mean_count, count.shape[1], len(fnids_country))
    table.loc[pd.IndexSlice[country_name,:,season_name],product_name] = string
print('Mean record years (# of districts)')
table

Mean record years (# of districts)


product                                Maize     Sorghum        Rice  \
country      admin  season                                             
Somalia      1990A2 Gu            23 (38/74)  19 (39/74)   11 (1/74)   
                    Deyr          22 (38/74)  19 (41/74)    5 (2/74)   
                    Gu-off         5 (26/74)    2 (3/74)           -   
                    Deyr-off       4 (27/74)    1 (2/74)           -   
Malawi       2007A2 Winter        12 (28/28)           -  10 (11/28)   
                    Main          39 (28/28)  25 (26/28)  24 (28/28)   
Kenya        2013A1 Annual        21 (46/47)   6 (46/47)   2 (23/47)   
                    Long          33 (47/47)   1 (44/47)           -   
                    Short          7 (45/47)   2 (35/47)           -   
Burkina Faso 2001A2 Main          34 (45/45)  34 (45/45)  33 (45/45)   
Mali         2001A1 Main            28 (7/9)    36 (7/9)    34 (7/9)   
Chad         2012A1 Main          25 (17/23)  29 (17/23)  22 (16/23)   
                    Cold-off               -  19 (17/23)           -   
South Africa 1994A1 Winter                 -           -           -   
                    Summer                 -    23 (9/9)           -   
Niger        2012A2 Main season   14 (42/74)  36 (69/74)  16 (21/74)   
Zambia       2012A2 Annual        33 (71/89)  18 (71/89)  14 (67/89)   
Angola       2008A1 Main          26 (18/18)   7 (16/18)  11 (10/18)   
Mozambique   2013A1 Main harvest  13 (10/11)  13 (10/11)   9 (10/11)   

product                               Millet       Wheat     Barley  \
country      admin  season                                            
Somalia      1990A2 Gu                     -           -          -   
                    Deyr                   -           -          -   
                    Gu-off                 -           -          -   
                    Deyr-off               -           -          -   
Malawi       2007A2 Winter          1 (2/28)    2 (7/28)          -   
                    Main          24 (26/28)   5 (16/28)          -   
Kenya        2013A1 Annual         6 (41/47)  23 (21/47)   3 (6/47)   
                    Long                   -           -          -   
                    Short                  -           -          -   
Burkina Faso 2001A2 Main          34 (45/45)           -          -   
Mali         2001A1 Main            36 (7/9)    13 (4/9)    4 (2/9)   
Chad         2012A1 Main          31 (18/23)   28 (1/23)          -   
                    Cold-off               -           -          -   
South Africa 1994A1 Winter                 -    39 (9/9)   11 (7/9)   
                    Summer                 -    16 (9/9)          -   
Niger        2012A2 Main season   37 (71/74)           -          -   
Zambia       2012A2 Annual        16 (70/89)   6 (34/89)  2 (13/89)   
Angola       2008A1 Main          10 (15/18)    1 (2/18)          -   
Mozambique   2013A1 Main harvest   8 (10/11)    1 (2/11)          -   

product                                Fonio Maize (White) Maize (Yellow)  
country      admin  season                                                 
Somalia      1990A2 Gu                     -             -              -  
                    Deyr                   -             -              -  
                    Gu-off                 -             -              -  
                    Deyr-off               -             -              -  
Malawi       2007A2 Winter                 -             -              -  
                    Main                   -             -              -  
Kenya        2013A1 Annual                 -             -              -  
                    Long                   -             -              -  
                    Short                  -             -              -  
Burkina Faso 2001A2 Main          18 (18/45)             -              -  
Mali         2001A1 Main            36 (5/9)             -              -  
Chad   

### Map of subnational level of FDW data (national)

In [6]:
fdw = gpd.read_file('./data/shapefile/country_fdw_grain_data_available.shp')
lims = fdw[fdw.continent == 'Africa'].total_bounds[[0,2,1,3]]
geojson = json.loads(fdw[['name','geometry']].to_json())
fdw = fdw[fdw['fdw_data'].notna()]
fdw['status'] = 'retrieved'
shape = gpd.read_file('./data/shapefile/adm_current.shp')
assert np.isin(shape['ADMIN0'].unique(),fdw['name']).all()
shape['level'] = shape['FNID'].apply(lambda x: 'admin-'+x[7])
admin_level = shape[['ADMIN0','level']].drop_duplicates()
for i, (country_name, level) in admin_level.iterrows():
    fdw.loc[fdw['name'] == country_name, 'status'] = level
poor_list = ['Uganda', 'South Sudan', 
    'Central African Republic', 'Zimbabwe', 'Guinea',
    "Côte d'Ivoire",
]
fdw.loc[fdw['name'].isin(poor_list), 'status'] = 'poor'

AssertionError: 

In [ ]:
# Mapping
fig = px.choropleth(
    data_frame=fdw,
    locations='name',
    color='status',
    geojson=geojson,
    featureidkey='properties.name',
    color_discrete_map={
        "retrieved": "LightSteelblue",
        "admin-1": "aquamarine",
        "admin-2": "aqua",
        "admin-3": "darkturquoise",
        "poor": "lightpink"
    },
    category_orders={"status": ["retrieved", "admin-1", "admin-2", "admin-3", "poor"]},
)
fig.update_traces(
    showscale=False,
    marker_line_width=1,
    marker_line_color='black',zmin=0, zmax=1,
)
fig.update_geos(visible=False, resolution=50,
                showcountries=True, countrycolor="LightSteelblue",
                lonaxis_range=lims[:2],
                lataxis_range=lims[2:4],
                showframe=False,
               )
fig.update_layout(
    font = {'family':'arial','size':15, 'color':'black'},
    width=660, height=600,
    margin={"r":0,"t":0,"l":0,"b":20},
    legend=dict(title='GSCD-FDW',font_size=15,x=1.02,y=1,xanchor='left',yanchor='top',bgcolor='rgba(0,0,0,0)'),
    font_size=14,
    dragmode=False
)
fig.add_annotation(
    xref='paper',yref='paper',
    x=0, y= -0.035,
    text='Source: FEWS NET Data Warehouse (FDW) crop production data',
    align="left",
    showarrow=False,
    font = {'family':'arial','size':15, 'color':'dimgrey'},
)
# fig.show()
fn_save = './figures/map_gscd_fdw_status_national.png'
fig.write_image(fn_save)
print('%s is saved.' % fn_save)

### Map of subnational level of FDW data (subnational)

In [ ]:
shape = gpd.read_file('./data/shapefile/adm_processed.shp')
shape['status'] = ''
shape.loc[shape['ADMIN0'].isin(fdw.loc[fdw['status'] == 'admin-1', 'name']),'status'] = 'admin-1'
shape.loc[shape['ADMIN0'].isin(fdw.loc[fdw['status'] == 'admin-2', 'name']),'status'] = 'admin-2'
shape.loc[shape['ADMIN0'].isin(fdw.loc[fdw['status'] == 'admin-3', 'name']),'status'] = 'admin-3'
shape.loc[shape['ADMIN0'].isin(fdw.loc[fdw['status'] == 'poor', 'name']),'status'] = 'poor'
fdw_copy = fdw.copy()
fdw_copy = fdw_copy.rename(columns={'name':'FNID'})
container = [shape, fdw_copy[fdw_copy['status'] == 'retrieved']]
shape = pd.concat(container, axis=0)
shape.geometry = shape.geometry.simplify(0.01)
geojson = json.loads(shape[['FNID','geometry']].to_json())

In [ ]:
# Mapping
fig = px.choropleth(
    data_frame=shape,
    locations='FNID',
    color='status',
    geojson=geojson,
    featureidkey='properties.FNID',
    color_discrete_map={
        "retrieved": "LightSteelblue",
        "admin-1": "aquamarine",
        "admin-2": "aqua",
        "admin-3": "darkturquoise",
        "poor": "lightpink"
    },
    category_orders={"status": ["retrieved", "admin-1", "admin-2", "admin-3", "poor"]},
)
fig.update_traces(
    showscale=False,
    marker_line_width=1,
    marker_line_color='black',zmin=0, zmax=1,
)
fig.update_geos(visible=False, resolution=50,
                showcountries=True, countrycolor="LightSteelblue",
                lonaxis_range=lims[:2],
                lataxis_range=lims[2:4],
                showframe=False,
               )
fig.update_layout(
    font = {'family':'arial','size':15, 'color':'black'},
    width=660, height=600,
    margin={"r":0,"t":0,"l":0,"b":20},
    legend=dict(title='GSCD-FDW',font_size=15,x=1.02,y=1,xanchor='left',yanchor='top',bgcolor='rgba(0,0,0,0)'),
    font_size=14,
    dragmode=False
)
fig.add_annotation(
    xref='paper',yref='paper',
    x=0, y= -0.035,
    text='Source: FEWS NET Data Warehouse (FDW) crop production data',
    align="left",
    showarrow=False,
    font = {'family':'arial','size':15, 'color':'dimgrey'},
)
# fig.show()
fn_save = './figures/map_gscd_fdw_status_subnational.png'
fig.write_image(fn_save)
print('%s is saved.' % fn_save)

### Somalia - Gu (table format)

In [ ]:
# for product_name in ['Maize', 'Sorghum']:
#     df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
#     df = df[
#         (df['country'] == 'Somalia') &
#         (df['product'] == product_name) &
#         (df['season_name'] == 'Gu')
#     ].reset_index(drop=True)
#     df = df.pivot_table(index='fnid',columns=['indicator','year'],values='value')
#     with pd.ExcelWriter('./public/somalia/adm_crop_production_table_somalia_gu_%s.xlsx' % product_name.lower()) as writer:  
#         df['production'].to_excel(writer, sheet_name='quantity_produced')
#         df['area'].to_excel(writer, sheet_name='harvested_area')
#         df['yield'].to_excel(writer, sheet_name='yield')

## Export to public data folder

In [ ]:
# # Weston's Google Drive
# shape = gpd.read_file('./data/shapefile/adm_current.shp')
# df = pd.read_hdf('./data/crop/adm_crop_production_ALL.hdf')
# df = df.merge(shape[['FNID','ADMIN0','ADMIN1','ADMIN2']], left_on='fnid', right_on='FNID')
# df = df.rename(columns={'ADMIN1':'admin1','ADMIN2':'admin2','season_name':'season'})
# df = df[['fnid','country','admin1','admin2','product','season','harvest_end','year','indicator','value']]
# df = df[
#     (df['season'].isin(['Gu','Deyr','Long','Short','Main'])) &
#     (df['product'].isin(['Maize','Wheat','Sorghum']))
# ].reset_index(drop=True)
# cps = df[['country','product','season']].drop_duplicates()
# print(cps)
# df_yield = df[df['indicator'] == 'yield']
# df_yield = df_yield.pivot_table(index=['fnid','country','admin1','admin2','product','season','harvest_end','indicator'], columns='year', values='value')
# df_yield = df_yield.astype(np.float32).round(3)
# fn_out = './public/grain_yield_africa.csv'
# df_yield.to_csv(fn_out)
# print("%s is saved." % fn_out)
# fn_out = './public/grain_pay_africa.csv'
# df.to_csv(fn_out)
# print("%s is saved." % fn_out)
# shape.to_file('./public/admin_africa.shp')
# print('./public/admin_africa.shp is saved.')